# Linear Regression

revision: dcfbda7

In [ ]:
import matplotlib.pyplot as plt
# PREAMBLE
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.optimize import check_grad

%matplotlib inline
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")
%load_ext autoreload
%autoreload 2

In [ ]:
# load all data
df = pd.read_csv('energy_summer_small.csv', sep=' ', index_col='Date')
df.index = pd.to_datetime(df.index)
print(df)

### Exploring the data

Before starting on any task, it is often useful to understand the data by visualizing it. In this example it can be done using a scatter plot since we only have two variables (temperature and load).

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x='temp', y='energy', data=df)

### Feature Scaling

Scale the temperature column such that it has zero mean and unit variance. It is best not to overwrite `df['temp']`, but to create a new column `df['temp normalized']`.
Compare this to figure 1.4 on page 5.

In [ ]:
df['temp normalized'] = 1  # insert your code here.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(16, 4))
sns.scatterplot(ax=ax[0], x='temp', y='energy', data=df)
sns.scatterplot(ax=ax[1], x='temp normalized', y='energy', data=df)

### Bias Term
In order to facilitate the vector notation $w^Tx$ we add a bias term to the vector $x$ i.e. an element with the value $1$.

In [ ]:
# add bias term
df['bias'] = 1
print(df)

In [ ]:
# export pandas dataframe to numpy array
X = df[['bias', 'temp normalized']].values
y = df['energy'].values
m, n = X.shape

print(f"Dimensions X: {X.shape}, y: {y.shape}")

In [ ]:
# Open mlis/regression/linearregression.py and implement the functions h and J.
from mlis.regression.linearregression import h, J

# test the hypothesis and cost function for w = [2,3]
w = np.array([2, 3])
# test the first 3 values calculated by h
np.testing.assert_array_almost_equal(h(w, X)[:3], [-0.76, -0.08, -0.08], decimal=2)
# test the correctness of J
np.testing.assert_array_almost_equal(J(w, X, y), 8.54, decimal=2)

Since $w$ is only two-dimensional, we can visualize $J(w)$ with a surface plot

In [ ]:
def plot_objective_contours(X, y, xlim=(0, 5), ylim=(-1.5, 2)):
    XX, YY = np.meshgrid(np.linspace(*xlim, 200), np.linspace(*ylim, 200))
    W = np.hstack([np.ravel(XX)[:, None], np.ravel(YY)[:, None]])
    ZZ = [J(w, X, y) for w in W]
    ZZ = np.array(ZZ).reshape(XX.shape)

    plt.contour(XX, YY, ZZ, levels=10)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel("$w_1$")
    plt.ylabel("$w_2$")


plot_objective_contours(X, y)

### Implementing gradient descent

Now that we have the hypothesis and the cost function we can implement the gradient descent routine. First, write a function $\nabla J(w)$ which returns the gradient of the cost function. The gradient is just a vector with all the partial derivatives
$$
\nabla J(w) = \bigg(\frac{\partial J(w)}{\partial w_1} , \dotsc, \frac{\partial J(w)}{\partial w_n} \bigg)
$$
The gradient of the least squares objective is
$$
\nabla J(w) = \frac{2}{m}X^{\intercal}(Xw-y)
$$
when written in matrix-form (compare this to the script).

In [ ]:
# Open mlis/regression/linearregression.py and implement the function dJ.
from mlis.regression.linearregression import dJ

# test the gradient for w = [2,3]
w = np.array([2, 3])
assert dJ(w, X, y).shape == (n,)  # we expect a vector of size (n,)
np.testing.assert_almost_equal(check_grad(J, dJ, w, X, y), 0.0, decimal=3)  # test the gradient

Implement the gradient descent update rule
$$
w \leftarrow w - \eta \nabla J(w)
$$

In order to inspect the performance of the algorithm it is necessary to store the weights $w$ at each iteration $t = 1,\dots,iters$.

In [ ]:
# Open mlis/regression/linearregression.py and implement the function gradientDescent

from mlis.regression.linearregression import gradientDescent

w0 = [1, 1]  # initial weights
eta = 0.05  # learning rate η
iters = 100  # number of iterations

ws = gradientDescent(dJ, X, y, w0, eta, iters)
# the last entry contains the final parameters found by gradient decent
print(f"best weights w: {ws[-1]}")

# testing the correctness of gradientDescent
np.testing.assert_array_almost_equal(ws[-1], [3.1168709, 0.27646542], decimal=4)  # test optimal weights

### Verifying Gradient Descent

A good way to verify that gradient descent is working correctly is to look at the values $J(w)$ and check that it is decreasing with each step.
Compare the following plot to figure 1.5 on page 6.

In [ ]:
def plot_progress(costs, log=True):
    with sns.axes_style("darkgrid"):
        _, ax = plt.subplots()
        ax.plot(np.arange(len(costs)), costs, 'k')
        ax.set_xlabel('Iterations')
        ax.set_ylabel('J(w)')
        if log:
            ax.set_yscale('log')
        return ax


plot_progress([J(w, X, y) for w in ws]);

In [ ]:
# show the path gradient descent took on the objective contours
plot_objective_contours(X, y)
plt.plot(ws[:, 0], ws[:, 1], '-ko');

Show the model that corresponds to the learned parameters.
Compare this to the plot in the lecture notes.

In [ ]:
def plot_fit(h, w, normalized=True):
    outputs = 'energy'
    inputs = 'temp normalized' if normalized else 'temp'
    hue = None

    t = np.linspace(df[inputs].min(), df[inputs].max(), 100)
    x = np.vstack([np.ones(t.shape), t]).T  # add bias term
    yh = np.squeeze(h(w, x))

    _, ax = plt.subplots()
    sns.scatterplot(ax=ax, x=inputs, y=outputs, hue=hue, data=df)
    ax.plot(t, yh, 'k', label='Prediction')


plot_fit(h, ws[-1])

### Omitting Feature Scaling

Plot the objective and gradient descent progress without feature scaling.

In [ ]:
X = df[['bias', 'temp', ]].values
y = df['energy'].values

# Try different values for eta and iters.
# With the following defaults, you will get numerical errors.
w0 = (4, 1)
ws = gradientDescent(dJ, X, y, w0, eta=0.001, iters=100)
print(f"calculated weights: {ws[-1]}")
print(f"optimal weights:    [ 0.3587, 0.1127]")

# plot objective contours
plot_objective_contours(X, y, xlim=[-1, 5], ylim=[-2, 2])

# plot only 100 weights
idx = np.linspace(0, len(ws) - 1, 100).astype(int)
plt.plot(ws[idx, 0], ws[idx, 1], '-ko')

# plot the objective values for the weights on a log scale
_, ax = plt.subplots()
ax.plot(idx, [J(w, X, y) for w in ws[idx, :]])
ax.set_yscale('log')

## Analytic Solution to Least Squares

We know that there exists a closed-form solution to linear regression which is given by
$$
w^{\star} = \big(X^TX \big)^{-1}X^Ty
$$
Using this formula does not require any feature scaling, and you will get an exact solution in one calculation: there is no "loop until convergence" like in gradient descent. However, it can only be used with the squared loss.

Important: In general, one should avoid calculating matrix inverses as this leads to numerical problems. Instead, use `np.linalg.solve` to solve the system of linear equations $X^TXw=X^Ty$.

In [ ]:
# Open mlis/regression/linearregression.py and implement the function least_squares

from mlis.regression.linearregression import least_squares

X = df[['bias', 'temp', ]].values
y = df['energy'].values
w = least_squares(X, y)

# testing the correctness of least_squares
np.testing.assert_array_almost_equal(least_squares(X, y), [0.3587, 0.1127], decimal=4)  # test optimal weights

# plot the model
plot_fit(h, w, normalized=False)

## Additional Features
We now use the additional input feature from the data set column `non_working`

In [ ]:
_, ax = plt.subplots()
sns.scatterplot(ax=ax, x='temp', y='energy', hue='non_working', data=df)

Create a column `df["is_workday"]`, which is `0` if the column `non_working` has entry `non-working` and 1` otherwise.

In [ ]:
# add the weekday column to our data frame
df["is_workday"] = True  # Your code here

In [ ]:
df["is_workday"] = (df["non_working"] == 'working') * 1

In [ ]:
# the scatter plot should look the same as above, just with a 0/1 legend
_, ax = plt.subplots()
sns.scatterplot(ax=ax, x='temp', y='energy', hue='is_workday', data=df)

Calculate the optimal weights for the model with the additional `non_working` feature using the closed-form solution.

In [ ]:
# insert your code here.
X = None
y = None
w = None
print(f"optimal weights w: {w}")
# expected is [0.28023641 0.25347537 0.10915125]

In [ ]:
X = df[['bias', 'is_workday', 'temp']].values
y = df['energy'].values
w = least_squares(X, y)

In [ ]:
np.testing.assert_almost_equal(least_squares(X, y), (0.2802, 0.2535, 0.1092), decimal=4)

Visualize the model fit by plotting the resulting predictions for both the weekend and weekday cases. Your plot should be identical to figure 2.2 on page 11

In [ ]:
# insert your code here.

In [ ]:
_, ax = plt.subplots()
sns.scatterplot(ax=ax, x='temp', y='energy', hue='is_workday', data=df)
t = np.linspace(df['temp'].min(), df['temp'].max(), 100)

# workday predictions
x = np.vstack([np.ones(t.shape), np.ones(t.shape), t]).T
y = np.squeeze(h(w, x))
plt.plot(t, y, 'r', label='Prediction')

# non-workday predictions
x = np.vstack([np.ones(t.shape), np.zeros(t.shape), t]).T
y = np.squeeze(h(w, x))

plt.plot(t, y, 'b', label='Prediction')

Create Features
$$
   x = \begin{bmatrix}
   \text{is_workday}\\
   1-\text{is_workday}\\
   \text{temp} \cdot \text{is_workday}\\
   \text{temp} \cdot (1-\text{is_workday})
   \end{bmatrix}
$$
and reproduce figure 2.3 on page 11.

In [ ]:
# insert your code here.

In [ ]:
X = np.vstack(
    (df['is_workday'], 1 - df['is_workday'], df['is_workday'] * df['temp'], (1 - df['is_workday']) * df['temp'])).T
y = df['energy'].values
w = least_squares(X, y)

_, ax = plt.subplots()
sns.scatterplot(ax=ax, x='temp', y='energy', hue='is_workday', data=df)
t = np.linspace(df['temp'].min(), df['temp'].max(), 100)

# workday predictions
x = np.vstack([np.ones(t.shape), t]).T
y = np.squeeze(h([w[0], w[2]], x))
plt.plot(t, y, 'r', label='Prediction')

# non-workday predictions
y = np.squeeze(h([w[1], w[3]], x))
plt.plot(t, y, 'b', label='Prediction')

# Loss Functions for Regression

The goal of this section is to compare the absolute and squared loss on a small data sets with an outlier.
First, we load the data a plot the squared loss result.

In [ ]:
df = pd.read_csv('energy_summer_small.csv', sep=' ', index_col='Date')
# add an outlier to the data set
df['outlier'] = 'normal'
outlier = pd.DataFrame(
    dict(
        Date='1900-01-01',
        temp=35,
        energy=2.7,
        outlier='outlier',
        weekday='Sunday',
        season='summer',
        non_working='working',
        split='train',
    ),
    index=[0]).set_index('Date')
df = pd.concat([df, outlier])

# normalize inputs
df['temp normalized'] = (df['temp'] - df['temp'].mean()) / df['temp'].std()
# add bias term
df['bias'] = 1

# data frame to matrix
X = df[['bias', 'temp normalized']].values
y = df['energy'].values

# run regression with squared loss
ws = gradientDescent(dJ, X, y, w0=np.zeros((2,)), eta=0.1, iters=100)
w_squared_loss = ws[-1]
plot_fit(h, w_squared_loss)

### Robust Regression

In [ ]:
# Open mlis/regression/robustregression.py and implement the functions J, dJ.
from mlis.regression.robustregression import J, dJ

w = np.array([2, 3])

# test the correctness of J
np.testing.assert_array_almost_equal(J(w, X, y), 2.27, decimal=2)

assert dJ(w, X, y).shape == (n,)  # we expect a vector of size (n,)
np.testing.assert_almost_equal(check_grad(J, dJ, w, X, y), 0.0, decimal=3)  # test the gradient

In [ ]:
# run regression with absolute loss loss
ws = gradientDescent(dJ, X, y, w0=np.zeros((2,)), eta=0.1, iters=100)
w_abs_loss = ws[-1]
plot_fit(h, w_abs_loss)